In [2]:
import numpy as np

class StepwiseMADS:
    def __init__(self, x0, bounds, delta0=1.0, delta_min=1e-3, alpha=2.0, gamma=0.5):
        self.x = np.array(x0)
        self.bounds = np.array(bounds)
        self.delta = delta0
        self.delta_min = delta_min
        self.alpha = alpha
        self.gamma = gamma
        self.mesh_size = delta0
        self.iteration = 0
        self.pending_eval = None
        self.best_value = None  # Store the best evaluation value
    
    def get_next_candidate(self):
        if self.pending_eval is not None:
            raise RuntimeError("Previous evaluation result is still pending.")
        
        directions = np.eye(len(self.x))
        candidates = [self.x + self.mesh_size * d for d in directions] + [self.x - self.mesh_size * d for d in directions]
        
        candidates = [np.clip(c, self.bounds[:, 0], self.bounds[:, 1]) for c in candidates]
        self.pending_eval = candidates.copy()
        return candidates
    
    def update_with_result(self, evaluations):
        if self.pending_eval is None:
            raise RuntimeError("No pending evaluations to update with.")
        
        best_idx = np.argmin(evaluations)
        best_candidate = self.pending_eval[best_idx]
        best_value = evaluations[best_idx]
        
        if self.best_value is None or best_value < self.best_value:
            self.x = best_candidate
            self.best_value = best_value
            self.mesh_size *= self.alpha
        else:
            self.mesh_size *= self.gamma
        
        self.mesh_size = max(self.mesh_size, self.delta_min)
        self.pending_eval = None
        self.iteration += 1
    
    def get_current_solution(self):
        return self.x


In [8]:
# Example Usage
def obj_func(x):
    a, b, c = x
    return (a - 0.5)**2 + (b - 15)**2 + (c - 1.5)**2  # Example function

bounds = np.array([[0, 1], [10, 20], [0, 3]])
mads = StepwiseMADS(x0=[0.9, 19, 0.1], bounds=bounds)
mads.best_value = obj_func(mads.x)  # Initialize best value with initial point evaluation
print("initial best value", mads.best_value)

for _ in range(20):  # Run for a few iterations
    candidates = mads.get_next_candidate()
    print("candidates:", candidates)
    evaluations = [float(obj_func(c)) for c in candidates]  # Simulated evaluation
    print("evaluations:", evaluations)
    mads.update_with_result(evaluations)
    print(f"Iter {_}candidates, evaluations", list(zip(candidates, evaluations)))
    print("current solution", mads.get_current_solution())
    print(f"Iteration {_}: Best solution {mads.get_current_solution()} with value {mads.best_value}")

initial best value 18.12
candidates: [array([ 1. , 19. ,  0.1]), array([ 0.9, 20. ,  0.1]), array([ 0.9, 19. ,  1.1]), array([ 0. , 19. ,  0.1]), array([ 0.9, 18. ,  0.1]), array([ 0.9, 19. ,  0. ])]
evaluations: [18.21, 27.12, 16.32, 18.21, 11.12, 18.41]
Iter 0candidates, evaluations [(array([ 1. , 19. ,  0.1]), 18.21), (array([ 0.9, 20. ,  0.1]), 27.12), (array([ 0.9, 19. ,  1.1]), 16.32), (array([ 0. , 19. ,  0.1]), 18.21), (array([ 0.9, 18. ,  0.1]), 11.12), (array([ 0.9, 19. ,  0. ]), 18.41)]
current solution [ 0.9 18.   0.1]
Iteration 0: Best solution [ 0.9 18.   0.1] with value 11.12
candidates: [array([ 1. , 18. ,  0.1]), array([ 0.9, 20. ,  0.1]), array([ 0.9, 18. ,  2.1]), array([ 0. , 18. ,  0.1]), array([ 0.9, 16. ,  0.1]), array([ 0.9, 18. ,  0. ])]
evaluations: [11.209999999999999, 27.12, 9.52, 11.209999999999999, 3.12, 11.41]
Iter 1candidates, evaluations [(array([ 1. , 18. ,  0.1]), 11.209999999999999), (array([ 0.9, 20. ,  0.1]), 27.12), (array([ 0.9, 18. ,  2.1]), 9.5